<a href="https://colab.research.google.com/github/kterashi/niigata_lecture_2024/blob/master/assignment_circuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Exercise] Implementation of Function, Hadamard Test, Quantum Fourier Transform

In [ ]:
# First, get all the necessary libraries from the copy and import packages
import os
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

In [ ]:
# Import everything
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit.visualization import plot_histogram

## Exercise 1: Adder circuit

Create input states to the adder ($a=5$, $b=6$) using $X$ gates, then implement $U$ and $V$. 

In [ ]:
input_digits = 3

# The number of bits in the circuit is the input digits x 2 + 2 (auxiliary bits)
circuit_width = 2 * input_digits + 2
qreg = QuantumRegister(circuit_width, name='q')
# Measure only bits where the added result is written
creg = ClassicalRegister(input_digits + 1, name='out')
circuit = QuantumCircuit(qreg, creg)

# Construct circuit tp create input states (a=5, b=6) using X gates
##################
### EDIT BELOW ###
##################

# for iq in [?, ?, ?, ..]:
#     circuit.x(iq)

##################
### EDIT ABOVE ###
##################

circuit.barrier()

# Apply U to qlow, qlow+1, qlow+2. Note the loop over the values set by range(0, n, 2).
for qlow in range(0, circuit_width - 2, 2):
    ##################
    ### EDIT BELOW ###
    ##################

    # Write circuit for U

    ##################
    ### EDIT ABOVE ###
    ##################

circuit.cx(circuit_width - 2, circuit_width - 1)

# Apply V to qlow, qlow+1, qlow+2. Note the loop over the values set by range(n-1, -1, -2).
for qlow in range(circuit_width - 4, -1, -2):
    ##################
    ### EDIT BELOW ###
    ##################

    # Write circuit for V

    ##################
    ### EDIT ABOVE ###
    ##################

# Measure [1st, 3rd, ...] qubits and write into classical register
circuit.measure(range(1, circuit_width, 2), creg)

circuit.draw('mpl')

In [ ]:
# Execute circuit with simulator
simulator = AerSimulator()
sampler = AerSampler()
shots = 100

circuit = transpile(circuit, backend=simulator)
job_result = sampler.run([circuit], shots=shots).result()
counts = job_result[0].data.out.get_counts()

plot_histogram(counts)

## Exercise 2: Idenity state vector using Hadamard test

We will examine the state \ket{\psi}$ created using the following circuit.

In [ ]:
# Size of the data register
data_width = 6

# Circuit to create the state |psi>
upsi = QuantumCircuit(data_width, name='psi')
upsi.x(0)
upsi.h(2)
upsi.cx(2, 3)
for itarg in range(data_width - 1, -1, -1):
    upsi.h(itarg)
    for ictrl in range(itarg - 1, -1, -1):
        power = ictrl - itarg - 1 + data_width
        upsi.cp((2 ** power) * 2. * np.pi / (2 ** data_width), ictrl, itarg)

for iq in range(data_width // 2):
    upsi.swap(iq, data_width - 1 - iq)

In Qiskit, we can convert a circuit of `QuantumCircuit` instance to a gate object with `to_gate()` method. Furthermore, if we apply `control(n)` to that gate, then the `n`-qubit controlled operation to the original circuit becomes possible. 

In [ ]:
upsi_gate = upsi.to_gate()
cupsi_gate = upsi_gate.control(1)

The $U^{-1}_k$ and its conversion to controlled gate are defined as a function with $k$.  

In [ ]:
def make_cukinv_gate(k):
    uk = QuantumCircuit(data_width, name=f'u_{k}')

    # Here unpackbits is used to get a binary representation of k
    # unpackbits takes an array of uint8, so the k is first converted to that
    k_bits = np.unpackbits(np.asarray(k, dtype=np.uint8), bitorder='little')
    # Get index of k_bits array where the corresponding bit is non-zero, and apply the X gate
    for idx in np.nonzero(k_bits)[0]:
        uk.x(idx)

    # Create the inverse circuit (though this is identical as the inverse operation of X is X itself)
    ukinv = uk.inverse()

    ukinv_gate = ukinv.to_gate()
    cukinv_gate = ukinv_gate.control(1)

    return cukinv_gate

A gate object can be added to `QuantumCircuit` object using `append()` function. When adding a controlled gate, the control bits correspond to the first n-bits of the target circuit. The `qargs` argument of `append()` can be used for that.

Execute the two kinds of Hadamard tests over $k$ from 0 though $2^n-1$, and determine the decomposition of $\ket{\psi}$ in the computational basis. 

In [ ]:
reg_data = QuantumRegister(data_width, name='data')
reg_test = QuantumRegister(1, name='test')
creg_test = ClassicalRegister(1, name='out')

# Circuits for real and imaginary parts are put into lists and passed to simulator 
circuits_re = []
circuits_im = []

ks = np.arange(2 ** data_width)

for k in ks:
    circuit_re = QuantumCircuit(reg_data, reg_test, creg_test)
    circuit_im = QuantumCircuit(reg_data, reg_test, creg_test)

    ##################
    ### EDIT BELOW ###
    ##################

    # Example of adding a controlled gate to circuit
    # circuit.append(cupsi_gate, qargs=([reg_test[0]] + reg_data[:]))

    ##################
    ### EDIT ABOVE ###
    ##################

    circuit_re.measure(reg_test, creg_test)
    circuit_im.measure(reg_test, creg_test)

    circuits_re.append(circuit_re)
    circuits_im.append(circuit_im)

# Execute circuit with simulator
simulator = AerSimulator()
sampler = AerSampler()
shots = 10000

circuits_re = transpile(circuits_re, backend=simulator)
circuits_im = transpile(circuits_im, backend=simulator)

job_result_re = sampler.run(circuits_re, shots=shots).result()
job_result_im = sampler.run(circuits_im, shots=shots).result()

# Array of state vectors
statevector = np.empty(2 ** data_width, dtype=np.complex128)

for k in ks:
    counts_re = job_result_re[k].data.out.get_counts()
    counts_im = job_result_im[k].data.out.get_counts()
    statevector[k] = (counts_re.get('0', 0) - counts_re.get('1', 0)) / shots
    statevector[k] += 1.j * (counts_im.get('0', 0) - counts_im.get('1', 0)) / shots

In [ ]:
plt.plot(ks, statevector.real, label='Re($c_k$)')
plt.plot(ks, statevector.imag, label='Im($c_k$)')
plt.xlabel('k')
plt.legend();

Compare the results and the state vectors obtained from state-vector simulator 

In [ ]:
sv_simulator = AerSimulator(method='statevector')

# Copy the original circuit and add save_statevector
circuit = upsi.copy()
circuit.save_statevector()

circuit = transpile(circuit, backend=sv_simulator)
statevector_truth = np.asarray(sv_simulator.run(circuit).result().data()['statevector'])

plt.plot(ks, statevector_truth.real, label='Re($c_k$) truth')
plt.plot(ks, statevector_truth.imag, label='Im($c_k$) truth')
plt.scatter(ks, statevector.real, label='Re($c_k$)')
plt.scatter(ks, statevector.imag, label='Im($c_k$)')
plt.xlabel('k')
plt.legend();

## Exercise 3: Generate probability distribution with the shape of cosine function

Create quantum circuit for which the probability of obtaining a bit string $k$ in the measurement is $\frac{1}{2}[1+\cos(8\pi k /2^5)]$.

In [ ]:
num_qubits = 5

circuit = QuantumCircuit(num_qubits)

##################
### EDIT BELOW ###
##################

# Set up a superposition of computational basis states

##################
### EDIT ABOVE ###
##################

# QFT circuit in the lecture

# Loop over target qubits from num_qubits - 1 to 0
for itarg in range(num_qubits - 1, -1, -1):
    # Apply Hadamard gate to each target qubit
    circuit.h(itarg)
    # Loop over control qubits from target - 1 to 0
    for ictrl in range(itarg - 1, -1, -1):
        # Apply controlled-P gate with the angle depending on the indices of target and control qubits
        power = ictrl - itarg - 1 + num_qubits
        circuit.cp((2 ** power) * 2. * np.pi / (2 ** num_qubits), ictrl, itarg)

    # Add barrier for visibility
    circuit.barrier()

# Flip the order of qubits at the end
for i in range(num_qubits // 2):
    circuit.swap(i, num_qubits - 1 - i)

circuit.measure_all()

# Use Sampler on the simulator
simulator = AerSimulator()
sampler = AerSampler()
shots = 100000

circuit = transpile(circuit, backend=simulator)
sim_job = sampler.run([circuit], shots=shots)
counts_dict = sim_job.result()[0].data.meas.get_counts()

# Convert measurement results to array for plot
counts = np.zeros(2 ** num_qubits)
for key, value in counts_dict.items():
    counts[int(key, 2)] = value
counts /= shots

# Make plots of the results and predictions
plt.scatter(np.arange(2 ** num_qubits), counts, label='observed')
x = np.linspace(0., 2 ** num_qubits, 400)
y = (1. + np.cos(8. * np.pi * x / 2 ** num_qubits)) / 2 ** num_qubits
plt.plot(x, y, label='target')
plt.legend();

## レポートとして提出するもの

- Exercise 1、2, 3の完成した回路のコード（EDIT BELOWからEDIT ABOVEの部分を埋める）
- シミュレーション結果で得られるプロット